# Install modules

In [ ]:
# !pip install datasets torch torchvision scikit-learn pandas numpy psutil

# Import modules

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from datasets import load_dataset
from torch.utils.data import IterableDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import os
import time
from PIL import Image
import psutil

# Focal Loss

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = (1 - pt) ** self.gamma * BCE_loss
        if self.alpha is not None:
            alpha_t = self.alpha * targets + (1 - self.alpha) * (1 - targets)
            F_loss = alpha_t * F_loss
        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

# Set up model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class CNNModel(nn.Module):
    def __init__(self, name='resnet18', num_classes=6):
        super(CNNModel, self).__init__()
        if name == 'resnet18':
            self.encoder = models.resnet18(pretrained=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(512, num_classes)
        elif name == 'resnet50':
            self.encoder = models.resnet50(pretrained=True)
            self.encoder.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(2048, num_classes)
        elif name == 'inception_v3':
            self.encoder = models.inception_v3(pretrained=True, aux_logits=True, transform_input=False)
            self.encoder.Conv2d_1a_3x3.conv = nn.Conv2d(1, 32, kernel_size=3, stride=2, bias=False)
            self.encoder.fc = nn.Identity()
            self.fc = nn.Linear(2048, num_classes)
        else:
            raise ValueError(f"Unsupported model name: {name}")
    
    def forward(self, x):
        output = self.encoder(x)
        if isinstance(output, tuple):
            output = output[0]
        return self.fc(output)

# Set up dataset

In [ ]:
class OLIVES_HF_Streaming(IterableDataset):
    def __init__(self, hf_dataset, transforms=None):
        self.dataset = hf_dataset
        self.transforms = transforms

    def __iter__(self):
        for item in self.dataset:
            try:
                image = item['Image'].convert("L")
                if self.transforms:
                    image = self.transforms(image)
                labels = torch.tensor([item[f'B{i}'] for i in range(1, 7)], dtype=torch.float)
                yield image, labels
            except Exception as e:
                print(f"Skipping invalid sample: {e}")
                continue

class RECOVERY_HF_Streaming(IterableDataset):
    def __init__(self, hf_dataset, transforms=None):
        self.dataset = hf_dataset
        self.transforms = transforms

    def __iter__(self):
        for item in self.dataset:
            try:
                image = item['Image'].convert("L")
                if self.transforms:
                    image = self.transforms(image)
                yield image
            except Exception as e:
                print(f"Skipping invalid test sample: {e}")
                continue

# Set loader function

In [ ]:
import torchvision.transforms as transforms
from datasets import load_dataset
from torch.utils.data import DataLoader

def set_loader(opt):
    mean = (0.1706,)
    std = (0.2112,)
    normalize = transforms.Normalize(mean=mean, std=std)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=opt.img_size, scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])

    val_transform = transforms.Compose([
        transforms.Resize((opt.img_size, opt.img_size)),
        transforms.ToTensor(),
        normalize,
    ])

    try:
        print("Loading dataset in streaming mode...")
        dataset = load_dataset("gOLIVES/OLIVES_Dataset", "biomarker_detection", streaming=True)
        print("Dataset loaded successfully.")

        train_dataset = OLIVES_HF_Streaming(dataset['train'].take(2000), transforms=train_transform)
        val_dataset = OLIVES_HF_Streaming(dataset['train'].take(400), transforms=val_transform)
        test_dataset = RECOVERY_HF_Streaming(dataset['test'], transforms=val_transform)

        train_loader = DataLoader(train_dataset, batch_size=opt.batch_size, num_workers=0)  # Removed shuffle=True
        val_loader = DataLoader(val_dataset, batch_size=opt.batch_size, num_workers=0)
        test_loader = DataLoader(test_dataset, batch_size=1, num_workers=0, drop_last=False)

        print("Data loaders created successfully.")
        return train_loader, val_loader, test_loader
    except Exception as e:
        print(f"Error in set_loader: {str(e)}")
        raise

# Helper functions

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def set_model(opt):
    model = CNNModel(name=opt.model, num_classes=opt.ncls)
    criterion = torch.nn.CrossEntropyLoss()  # Replace with your criterion, e.g., FocalLoss if used
    model = model.to(opt.device)
    criterion = criterion.to(opt.device)
    return model, criterion

def set_optimizer(opt, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=opt.learning_rate, weight_decay=opt.weight_decay)
    return optimizer

def save_model(model, optimizer, opt, epoch, save_file):
    state = {'opt': opt.__dict__, 'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
    torch.save(state, save_file)

def adjust_learning_rate(opt, optimizer, epoch):
    lr = opt.learning_rate
    steps = sum(epoch > e for e in opt.lr_decay_epochs)
    if steps > 0:
        lr = lr * (opt.lr_decay_rate ** steps)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def train_supervised(train_loader, model, criterion, optimizer, epoch, opt):
    model.train()
    losses = AverageMeter()
    for idx, (image, labels) in enumerate(train_loader):
        images = image.to(opt.device)
        labels = labels.to(opt.device)
        bsz = labels.shape[0]
        output = model(images)
        loss = criterion(output, labels)
        losses.update(loss.item(), bsz)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (idx + 1) % opt.print_freq == 0:
            print(f'Train: [{epoch}][{idx + 1}]\tLoss {losses.avg:.4f}')
    return losses.avg

def sample_evaluation(val_loader, model, opt):
    model.eval()
    out_list, label_list = [], []
    with torch.no_grad():
        for idx, (image, labels) in enumerate(val_loader):
            images = image.to(opt.device)
            labels = labels.to(opt.device)
            output = model(images)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.detach().cpu().numpy())
            label_list.append(labels.detach().cpu().numpy())
            if idx >= 50:
                break
    out_array = np.concatenate(out_list, axis=0)
    label_array = np.concatenate(label_list, axis=0)
    f = f1_score(label_array, out_array, average='macro')
    print(f'Validation F1 Score (macro): {f:.4f}')
    return f

def submission_generate(test_loader, model, opt):
    model.eval()
    out_list = []
    with torch.no_grad():
        for idx, image in enumerate(test_loader):
            images = image.to(opt.device)
            output = model(images)
            output = torch.round(torch.sigmoid(output))
            out_list.append(output.squeeze().detach().cpu().numpy())
    out_submission = np.array(out_list)
    df = pd.DataFrame(out_submission, columns=['B1', 'B2', 'B3', 'B4', 'B5', 'B6'])
    df.to_csv('submission.csv', index=False)
    print("Submission file 'submission.csv' generated.")

# Configuration options

In [ ]:
import os

class Opt:
    def __init__(self):
        self.print_freq = 10
        self.save_freq = 50
        self.batch_size = 4
        self.num_workers = 0
        self.epochs = 20
        self.device = 'cpu'
        self.learning_rate = 0.0001
        self.lr_decay_epochs = [5, 8]
        self.lr_decay_rate = 0.1
        self.weight_decay = 1e-4
        self.ncls = 6
        self.model = 'inception_v3'  # Use InceptionNet V3
        self.dataset = 'OLIVES'
        self.save_folder = './save/OLIVES_models'
        
        # Define image sizes for different models
        model_img_sizes = {
            'resnet18': 224,
            'resnet50': 224,
            'inception_v3': 299,
        }
        self.img_size = model_img_sizes[self.model]
        
        self.model_name = f'{self.model}_lr_{self.learning_rate}_decay_{self.weight_decay}_bsz_{self.batch_size}'
        self.save_folder = os.path.join(self.save_folder, self.model_name)
        if not os.path.isdir(self.save_folder):
            os.makedirs(self.save_folder)

opt = Opt()

# Debug dataset (optional)

In [ ]:
"""
import psutil
import sys

# Check memory usage
print(f"Memory usage: {psutil.virtual_memory().percent}%")
print(f"Available memory: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

# Inspect dataset sample
try:
    dataset = load_dataset("gOLIVES/OLIVES_Dataset", "biomarker_detection", streaming=True)
    sample = next(iter(dataset['train']))
    print("Sample keys:", sample.keys())
    print("Image type:", type(sample['Image']))
    print("Biomarkers:", [sample[f'B{i}'] for i in range(1, 7)])
except Exception as e:
    print(f"Error inspecting dataset: {str(e)}")
"""

# Check memory usage (optional)

In [ ]:
"""
import psutil
import torch
import time

def print_memory_usage():
    process = psutil.Process()
    mem = process.memory_info().rss / (1024 ** 3)  # GB
    total_mem = psutil.virtual_memory().total / (1024 ** 3)
    used_mem = psutil.virtual_memory().used / (1024 ** 3)
    print(f"Process memory: {mem:.2f} GB")
    print(f"System memory: {used_mem:.2f}/{total_mem:.2f} GB ({psutil.virtual_memory().percent}%)")
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.memory_allocated() / (1024 ** 3)
        gpu_total = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        print(f"GPU memory: {gpu_mem:.2f}/{gpu_total:.2f} GB")

# Test memory before loading
print("Before loading dataset:")
print_memory_usage()

# Test data loading (mimic Cell 5 partially)
try:
    print("\nTesting data loading...")
    dataset = load_dataset("gOLIVES/OLIVES_Dataset", "biomarker_detection", streaming=True)
    train_iter = iter(dataset['train'])
    for i in range(10):  # Load 10 samples
        sample = next(train_iter)
        print_memory_usage()
        time.sleep(0.1)  # Slow down to observe
except Exception as e:
    print(f"Error during data loading test: {str(e)}")
"""

# Main function

In [ ]:
import psutil

def print_memory_usage():
    process = psutil.Process()
    mem = process.memory_info().rss / (1024 ** 3)
    total_mem = psutil.virtual_memory().total / (1024 ** 3)
    used_mem = psutil.virtual_memory().used / (1024 ** 3)
    print(f"Process memory: {mem:.2f} GB")
    print(f"System memory: {used_mem:.2f}/{total_mem:.2f} GB ({psutil.virtual_memory().percent}%)")

def main():
    print("Before loading data:")
    print_memory_usage()
    train_loader, val_loader, test_loader = set_loader(opt)
    print("Data loaders created.")
    print_memory_usage()

    model, criterion = set_model(opt)
    print(f"Model {opt.model} initialized on {opt.device}.")
    print_memory_usage()

    optimizer = set_optimizer(opt, model)

    best_f1 = 0
    patience = 10  # Increased from 5 to 10
    no_improve = 0

    for epoch in range(1, opt.epochs + 1):
        print(f"Before epoch {epoch}:")
        print_memory_usage()
        adjust_learning_rate(opt, optimizer, epoch)
        train_loss = train_supervised(train_loader, model, criterion, optimizer, epoch, opt)
        print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}')
        f1 = sample_evaluation(val_loader, model, opt)
        
        if f1 > best_f1:
            best_f1 = f1
            no_improve = 0
            save_file = os.path.join(opt.save_folder, 'best.pth')
            save_model(model, optimizer, opt, epoch, save_file)
            print(f"Best model saved with F1: {best_f1:.4f}")
        else:
            no_improve += 1
            if no_improve >= patience:
                print("Early stopping triggered.")
                break
        print_memory_usage()
    
    print(f"Best F1: {best_f1}")
    submission_generate(test_loader, model, opt)
    save_file = os.path.join(opt.save_folder, 'last.pth')
    save_model(model, optimizer, opt, opt.epochs, save_file)
    print(f"Model saved to {save_file}")

main()